# NOAA Global hourly data preprocessing
This notebook serves as experiment of preprocessing [NOAA data](https://www.ncei.noaa.gov/data/global-hourly/archive/csv/).

Data documentation is [here](https://www.ncei.noaa.gov/data/global-hourly/doc/isd-format-document.pdf)

Data sample is [here](https://www.ncei.noaa.gov/data/global-hourly/doc/)

In [1]:
import pandas as pd
import numpy as np

In [2]:
filename = '2020_72494023234.csv'
filepath = '../Data/72494023234/'+filename
df_raw = pd.read_csv(filepath)

#print(df_raw.columns)
df_raw.head()
#df_raw.columns

/opt/anaconda3/envs/aviation/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3524: DtypeWarning: Columns (37,74) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,STATION,DATE,SOURCE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,CALL_SIGN,QUALITY_CONTROL,...,OC1,OD1,OE1,OE2,OE3,RH1,RH2,RH3,REM,EQD
0,72494023234,2020-01-01T00:00:00,4,37.6197,-122.3647,2.4,"SAN FRANCISCO INTERNATIONAL AIRPORT, CA US",FM-12,99999,V020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SYN08072494 32966 41004 10133 20094 30165 4019...,NaN
1,72494023234,2020-01-01T00:56:00,7,37.6197,-122.3647,2.4,"SAN FRANCISCO INTERNATIONAL AIRPORT, CA US",FM-15,KSFO,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MET10512/31/19 16:56:03 METAR KSFO 010056Z 320...,D01 0ADE726
2,72494023234,2020-01-01T01:56:00,7,37.6197,-122.3647,2.4,"SAN FRANCISCO INTERNATIONAL AIRPORT, CA US",FM-15,KSFO,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MET10512/31/19 17:56:03 METAR KSFO 010156Z 290...,D01 0ADE726
3,72494023234,2020-01-01T02:56:00,7,37.6197,-122.3647,2.4,"SAN FRANCISCO INTERNATIONAL AIRPORT, CA US",FM-15,KSFO,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MET11012/31/19 18:56:03 METAR KSFO 010256Z 280...,D01 0ADE726
4,72494023234,2020-01-01T03:56:00,7,37.6197,-122.3647,2.4,"SAN FRANCISCO INTERNATIONAL AIRPORT, CA US",FM-15,KSFO,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MET10412/31/19 19:56:03 METAR KSFO 010356Z 290...,D01 0ADE726


In [ ]:
def ParseDF(df):
    # Select columns we care
    source_majority = df_raw['SOURCE'].mode()[0]
    df = df[df['SOURCE'] == source_majority]
    
    # PARSE
    # Split DATE
    datetime = pd.to_datetime(df['DATE'])
    df['DATE_year'] = datetime.dt.year
    df['DATE_month'] = datetime.dt.month
    df['DATE_day'] = datetime.dt.day
    df['DATE_hour'] = datetime.dt.hour
    df['DATE_year_month_day'] = df[['DATE_year', 'DATE_month', 'DATE_day']].apply(tuple, axis=1)
    
    columns_parse = ['WND','CIG','VIS','TMP','DEW']
    # Split Wind WND data
    df[['WND_angle','WND_angle_quality','WND_wind_observation','WND_speed','WND_speed_quality']] = df.WND.str.split(',', expand=True)
    # Split CIG data
    df[['CIG_ceiling_height', 'CIG_ceiling_quality','CIG_determ', 'CIG_CAVOK']] = df.CIG.str.split(',', expand=True)
    # Split VIS data
    df[['VIS_distance','VIS_distance_quality', 'VIS_observation','VIS_observation_quality']] = df.VIS.str.split(',', expand=True)
    # Split TMP data
    df[['TMP_temp_celsius','TMP_temp_quality']] = df.TMP.str.split(',', expand=True)
    # SPlit DEW data
    df[['DEW_temp_celsius','DEW_quality']] = df.DEW.str.split(',', expand=True)
    # Split SLP data
    df[['SLP_hectopascal','SLP_quality']] = df.SLP.str.split(',', expand=True)
    
    def getAT(input):
        #print(type(input))
        # individual data look like: "AU,13,BR ,5", or "none"
        if type(input) == str:
            try:
                return int(input.split(',')[1])
            except:
                return 0
        else:
            return 0
    
    def thunder_check(array, check=3):
        if check in array:
            return 1
        else:
            return 0
        
    def tornado_check(array, check=10):
        if check in array:
            return 1
        else:
            return 0
        
    def checkBadWeather(row):
        return 1 if np.sum(row)>0 else 0
    
    # AT
    df['AT1'] = df['AT1'].apply(getAT)
    df['AT2'] = df['AT2'].apply(getAT)
    df['AT3'] = df['AT3'].apply(getAT)
    #df['AT4'] = df['AT4'].apply(getAT)
    df['AT'] = list(df[['AT1','AT2','AT3']].to_numpy())
    df['AT_binary'] = df['AT'].apply(checkBadWeather)
    df['AT_thunder'] = df['AT'].copy()
    df['AT_thunder'] = df['AT_thunder'].apply(thunder_check)
    df['AT_tornado'] = df['AT'].copy()
    df['AT_tornado'] = df['AT_tornado'].apply(tornado_check)

    # Drop parsed raw data
    df.drop(columns_parse, axis=1, inplace=True)
    return df

def KeepRelevant(df):
    columns = ['STATION', 'DATE','DATE_year','DATE_month','DATE_day','DATE_hour','DATE_year_month_day', 'LATITUDE',
               'LONGITUDE', 'ELEVATION', 'WND_angle', 'WND_speed', 'CIG_ceiling_height', 'VIS_distance',
               'TMP_temp_celsius', 'DEW_temp_celsius','SLP_hectopascal','AT','AT_thunder','AT_tornado','AT_binary']
    df = df[columns]
    return df

def ReplaceMissingNumericWithNeighbor(df, column, to_replace=[999,9999,99999,999999], value=np.nan, rescale=1):
    '''Replace Missing values with NaN, Fill with method='ffill'''
    df[column] = df[column].astype(int)
    df[column].replace(to_replace=to_replace, value=np.nan, inplace=True)
    # Fill NaN
    df[column] = df[column].fillna(method='ffill')
    df[column] = df[column].fillna(method='bfill')
    
    if rescale != 1:
        df[column] = df[column]/rescale
        
    return df

# FIll thunder for all day
def Fill_thunder_day(df):    
    month_day_s = list(df[df['AT_thunder']==1]['DATE_year_month_day'])
    df.loc[df['DATE_year_month_day'].isin(month_day_s), 'AT_thunder']=1
    return df

# FIll thunder for all day
def Fill_tornado_day(df):    
    month_day_s = list(df[df['AT_tornado']==1]['DATE_year_month_day'])
    df.loc[df['DATE_year_month_day'].isin(month_day_s), 'AT_tornado']=1
    return df

def Fill_AT_binary_day(df):
    month_day_s = list(df[df['AT_binary']==1]['DATE_year_month_day'])
    df.loc[df['DATE_year_month_day'].isin(month_day_s), 'AT_binary']=1
    return df


def DataframePreprocessingPipeline(df):
    # Parse
    df = ParseDF(df)
    df = KeepRelevant(df)
    
    # Replace & Rescale
    df = ReplaceMissingNumericWithNeighbor(df, 'WND_angle')
    df = ReplaceMissingNumericWithNeighbor(df, 'WND_speed', rescale=10)
    df = ReplaceMissingNumericWithNeighbor(df, 'CIG_ceiling_height')
    df = ReplaceMissingNumericWithNeighbor(df, 'VIS_distance')
    df = ReplaceMissingNumericWithNeighbor(df, 'TMP_temp_celsius',rescale=10)
    df = ReplaceMissingNumericWithNeighbor(df, 'DEW_temp_celsius', rescale=10)
    df = ReplaceMissingNumericWithNeighbor(df, 'SLP_hectopascal', rescale=10)
    
    df = Fill_thunder_day(df)
    df = Fill_tornado_day(df)
    df = Fill_AT_binary_day(df)
    
    return df


In [ ]:
df = DataframePreprocessingPipeline(df_raw)
df

In [ ]:
df_exp = df.copy()
df_exp = df_exp.drop(['DATE','STATION','DATE_year_month_day','AT'], axis=1)

In [ ]:
y = df_exp['AT_binary']
X = df_exp.drop('AT_binary',axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=False)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

model = DecisionTreeClassifier(random_state=0).fit(X_train,y_train)
accuracy_score(y_test, model.predict(X_test))

In [ ]:
print('Majority:', 1 - sum(df_exp['AT_binary'])/len(df_exp['AT_binary']))

# Output
Save processed data

In [ ]:
filepath = filepath = '../DataProcessed/'+filename
df.to_csv(filepath, index=False)